# Challenge Data Science - Setembro 2022

**Identificar pacientes com alto risco cardiovascular**

Autor: Jorge Chamby Diaz
_________

## O Problema

As doenças cardiovasculares (DCVs) são a causa número 1 de morte em todo o mundo, levando cerca de 17,9 milhões de vidas a cada ano, o que representa 31% de todas as mortes em todo o mundo. Quatro de cada 5 mortes por CVDs são devido a ataques cardíacos e derrames, e um terço dessas mortes ocorre prematuramente em pessoas com menos de 70 anos de idade. 

Pessoas com doença cardiovascular ou com alto risco cardiovascular (devido à presença de um ou mais fatores de risco, como hipertensão, diabetes, hiperlipidemia ou doença já estabelecida) precisam de detecção e gerenciamento precoces, onde um modelo de aprendizado de máquina pode ser de grande ajuda.

## Os dados

A insuficiência cardíaca é um evento comum causado por DCVs e este conjunto de dados contém 11 características que podem ser usadas para prever uma possível doença cardíaca:

- Medidas de 11 variáveis que caracterizam cada amostra (as features do problema):
<br><br>
    - 1 - Age: idade do paciente (anos)
    - 2 - Sex: sexo do paciente (M: Masculino, F: Feminino)
    - 3 - ChestPainType: tipo de dor no peito (TA: Angina Típica, ATA: Angina Atípica, NAP: Dor Não Anginosa, ASY: Assintomática)
    - 4 - RestingBP: pressão arterial de repouso (mm Hg)
    - 5 - Cholesterol: colesterol sérico (mm/dl)
    - 6 - FastingBS: glicemia em jejum (1: se FastingBS > 120 mg/dl, 0: caso contrário)
    - 7 - RestingECG: resultados do eletrocardiograma de repouso (Normal: normal, ST: com anormalidade da onda ST-T, LVH: mostrando provável ou definitiva hipertrofia ventricular esquerda pelos critérios de Estes)
    - 8 - MaxHR: frequência cardíaca máxima alcançada (Valor numérico entre 60 e 202)
    - 9 - ExerciseAngina: angina induzida por exercício (Y: Sim, N: Não)
    - 10 - Oldpeak: pico antigo = ST (Valor numérico medido em depressão)
    - 11 - ST_Slope: a inclinação do segmento ST do exercício de pico (Up: upsloping, Flat: flat, Down: downsloping)    
<br><br>
- Além disso, há a variável resposta que no caso é uma variável binária:
<br><br>
    - 12 - HeartDisease: classe de saída (1: doença cardíaca, 0: normal)

Para maiores informações sobre a coleta e origem dos dados, veja a página do dataset no repositório UCI machine learning repository, [disponível aqui!](https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/) 

**Para baixar os conjuntos de dados, podem acessar diretamente a este [link](https://github.com/jchambyd/Datasets/tree/main/Heart-disease)**

## O Challenge

Suponha que você é um cientista de dados que trabalha na área de *bioinformatica* de uma clínica especializada no tratamento de doenças cardiovasculares. Nos últimos meses a empresa tem recebido uma grande quantidade de pacientes (o que é muito bom!), permitindo gerar um histórico de todos os pacientes que foram para consulta e saíram com um diagnóstico. Para simplificar as coisas, o diagnóstico foi reduzido a duas possíveis respostas: 'positivo' ou 'negativo'. Positivo significa que o paciente apresenta alguma doença cardiovascular, e negativo significa que a pessoa está livre de doenças cardiovasculares. A empresa pede para você construir um modelo preditivo que permita, a partir de um conjunto prévio de exames feitos sobre um paciente novo, identificar se o paciente possui ou não uma doença cardiovascular.

Com este modelo, os médicos vão ter uma ferramenta de apoio muito útil para realizar o diagnóstico de doenças cardiovasculares para novos pacientes que a clínica receba no futuro.

Dentro deste contexto, seu objetivo como cientista de dados é claro:

> Agregar valor ao negócio, explorando os dados que você tem à disposição.

Na primeira sprint do projeto, você e outros colegas do time de ciência de dados chegaram na seguinte _TO-DO list_ para o projeto. Algumas atividades já foram executadas nas primeiras semanas de trabalho (o time é muito bom!), mas agora ainda restam algumas atividades muito importantes (e divertidas!) a serem feitas:

- [x] Ingestão dos dados e detalhada análise exploratória
- [x] Formulação do problema
- [ ] Primeiro modelo baseline
- [ ] Iterações pelo ciclo de modelagem
- [ ] Compílação dos resultados para o negócio
- [ ] Comunicação dos resultados

Bom trabalho, e divirta-se! :D